#### Recap to Exponentially Weighted Moving Average:-

- Used a single parameter $\alpha$ as the smoothing parameter
- Didn't capture other parameters/factors like seasonality and trend

#### Holt Winter's Methods

- Comprises of the forecast equation with 3 additional smoothing equations:-
  - One for the level ($l_t$)
  - One for the trend component($b_t$)
  - One for the seasonal component ($s_t$)
  - Each equation (of the 3 above) has a corresponding smoothing parameter: $\alpha, \beta, \gamma$
- Two varying methods depending on how the seasonal component varies:-
  - additive method for constant seasonal variations
  - multiplicative method when seasonal variations change proportional to the level (average over a time period) of the series.

<div style="line-height:0px">
<h4> Single Exponential Smoothing (Same as EWMA in pandas)
<h6> ~ The base method for Double and Triple exponential smoothing using Holt-Winter's Methods
</div>

$y_0 = x_0$ </br>
$y_t = (1-\alpha)y_{t-1} + \alpha x_t$

#### Double Exponential Smoothing

- Holt's Method
- Strategy: To introduce a smoothing parameter $\beta$ that addresses trend

$l_t = (1-\alpha)l_{t-1} + \alpha x_t$ &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; [level]</br>    
$b_t = (1-\beta)b_{t-1} + \beta (l_t-l_{t-1})$ &emsp;&emsp;&emsp;&emsp;[trend] </br>
$y_t = l_t + b_t$ &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&nbsp;&emsp;[fitted model]


- Because seasonal fluctuations are **not** considered, the forecasted model is a straight sloped line.
- Just shows the general trend upwards/downwards/flat.

#### Triple Exponential Smoothing:-

- Introduces a $\gamma$ parameter as a smoothing factor that addresses seasonality.

$l_t = (1-\alpha)l_{t-1} + \alpha x_t$ </br>
$b_t = (1-\beta)b_{t-1} + \beta (l_t-l_{t-1})$ </br>
$c_t = (1-\gamma)c_{t-L}+\gamma (x_t-l_{t-1}-b_{t-1})$ </br>

$y_t = (l_t + b_t) c_t$

Here L is the number of divisions per cycle. So if a monthly data has a repeating pattern every year/12 months, L = 12.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [3]:
airline = pd.read_csv('airline.csv', index_col='Month', parse_dates=True)

In [4]:
airline.dropna(inplace=True)

In [12]:
airline.index

DatetimeIndex(['1949-01-01', '1949-02-01', '1949-03-01', '1949-04-01',
               '1949-05-01', '1949-06-01', '1949-07-01', '1949-08-01',
               '1949-09-01', '1949-10-01',
               ...
               '1960-03-01', '1960-04-01', '1960-05-01', '1960-06-01',
               '1960-07-01', '1960-08-01', '1960-09-01', '1960-10-01',
               '1960-11-01', '1960-12-01'],
              dtype='datetime64[ns]', name='Month', length=144, freq=None)

In [13]:
airline.index.freq = 'MS'

In [14]:
airline.index

DatetimeIndex(['1949-01-01', '1949-02-01', '1949-03-01', '1949-04-01',
               '1949-05-01', '1949-06-01', '1949-07-01', '1949-08-01',
               '1949-09-01', '1949-10-01',
               ...
               '1960-03-01', '1960-04-01', '1960-05-01', '1960-06-01',
               '1960-07-01', '1960-08-01', '1960-09-01', '1960-10-01',
               '1960-11-01', '1960-12-01'],
              dtype='datetime64[ns]', name='Month', length=144, freq='MS')

In [16]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [18]:
span = 12
alpha = 2/(span+1)

In [19]:
airline['EWMA'] = airline['Thousands of Passengers'].ewm(alpha=alpha, adjust=False).mean()

In [20]:
airline.head()

,Thousands of Passengers,EWMA
Month,,
1949-01-01,112,112.000000
1949-02-01,118,112.923077
1949-03-01,132,115.857988
1949-04-01,129,117.879836
1949-05-01,121,118.359861


In [22]:
model = SimpleExpSmoothing(airline['Thousands of Passengers'])
fit = model.fit(smoothing_level=alpha, optimized=False)

In [32]:
airline['SES12'] = fit.fittedvalues.shift(-1)

In [33]:
airline.head()

,Thousands of Passengers,EWMA,SES12
Month,,,
1949-01-01,112,112.000000,112.000000
1949-02-01,118,112.923077,112.923077
1949-03-01,132,115.857988,115.857988
1949-04-01,129,117.879836,117.879836
1949-05-01,121,118.359861,118.359861


In [34]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [35]:
modelExp = ExponentialSmoothing(airline['Thousands of Passengers'], trend='add')
fitExp = modelExp.fit()

In [37]:
fitExp.fittedvalues

Month
1949-01-01    120.527273
1949-02-01    114.102394
1949-03-01    120.040657
1949-04-01    134.001539
1949-05-01    131.085845
                 ...    
1960-08-01    623.656106
1960-09-01    608.168786
1960-10-01    510.571382
1960-11-01    463.313463
1960-12-01    392.424879
Freq: MS, Length: 144, dtype: float64